In [1]:
import torch
import random
import numpy as np
import regex

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def split_train_val_test(df, props=[.8, .1, .1]):
    assert round(sum(props), 2) == 1 and len(props) >= 2
    train_df, test_df, val_df = None, None, None

    train_size = int(props[0] * len(df))
    val_size =  train_size + int(props[1] * len(df))
    test_size =val_size + int(props[2] * len(df)) 
    train_df = df.iloc[0:train_size]
    val_df = df.iloc[train_size:val_size]
    test_df = df.iloc[val_size:test_size]
    
    return train_df, val_df, test_df

In [3]:
import gensim.downloader as api

def download_embeddings(fasttetxt):
    # https://fasttext.cc/docs/en/english-vectors.html
    if fasttetxt:
      wv = api.load("fasttext-wiki-news-subwords-300")
    else:
      
      wv = api.load("word2vec-google-news-300")
      print("\nLoading complete!\n" +
            "Vocabulary size: {}".format(len(wv.vocab)))
    return wv


In [4]:
# Opening and preprocessing input file
import gensim.models
import pandas as pd
import nltk
nltk.download('punkt')
from tqdm import tqdm
from preprocess import clean_text

data = pd.read_csv('train.csv', quotechar='"')
data.sample(frac=1)


# to convert authors into numbers
author_to_number = {
    'EAP': 0,
    'HPL': 1,
    'MWS': 2
    
}

# lowercase, removing punctuation and tookenize sentences. Converting labels to int
training_text = ""
for i in range(len(data)):

    data['text'][i] = nltk.word_tokenize(regex.sub(r'[^\w\s]', '',data['text'][i].lower()))
    data['author'][i] = author_to_number[data['author'][i]]

print(data[0:10])
print(len(data))

from dataset import *

# Splitting dataset and generating vocab
train_df, val_df, test_df = split_train_val_test(data)
train_vocab, reversed_vocab = generate_vocab_map(train_df)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
        id                                               text author
0  id26305  [this, process, however, afforded, me, no, mea...      0
1  id17569  [it, never, once, occurred, to, me, that, the,...      1
2  id11008  [in, his, left, hand, was, a, gold, snuff, box...      0
3  id27763  [how, lovely, is, spring, as, we, looked, from...      2
4  id12958  [finding, nothing, else, not, even, gold, the,...      1
5  id22965  [a, youth, passed, in, solitude, my, best, yea...      2
6  id09674  [the, astronomer, perhaps, at, this, point, to...      0
7  id13515  [the, surcingle, hung, in, ribands, from, my, ...      0
8  id19322  [i, knew, that, you, could, not, say, to, your...      0
9  id00912  [i, confess, that, neither, the, structure, of...      2
19579


In [25]:
DOWNLOAD = True
# Use fastext or word2vec
FASTTEXT = False
WINDOW_SIZE = 5

EMBEDDING_DIM = 300
HIDDEN_DIM = 128
NUM_LAYERS = 1
BIDIRECTIONAL = True


In [6]:
# Downloading or generating word2vec embeddings

if DOWNLOAD:
    model = download_embeddings(FASTTEXT)
else:
    if FASTTEXT:
        model = gensim.models.FastText(sentences=train_df['text'], size=EMBEDDING_DIM, window=WINDOW_SIZE)
    else:
        model = gensim.models.Word2Vec(sentences=train_df['text'], size=EMBEDDING_DIM, window=WINDOW_SIZE)
                        

[==================================================] 100.0% 1662.8/1662.8MB downloaded

Loading complete!
Vocabulary size: 3000000


In [7]:
from dataset import HeadlineDataset
from torch.utils.data import RandomSampler

train_dataset = HeadlineDataset(train_vocab, train_df,model.wv, FASTTEXT)
val_dataset = HeadlineDataset(train_vocab, val_df,model.wv, FASTTEXT)
test_dataset = HeadlineDataset(train_vocab, test_df,model.wv, FASTTEXT)

# Now that we're wrapping our dataframes in PyTorch datsets, we can make use of PyTorch Random Samplers.
train_sampler = RandomSampler(train_dataset)
val_sampler = RandomSampler(val_dataset)
test_sampler = RandomSampler(test_dataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [22]:
from torch.utils.data import DataLoader
from dataset import collate_fn
BATCH_SIZE = 16
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, collate_fn=collate_fn)
val_iterator = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)

for x, y in test_iterator:
    print(x,y)
    break

tensor([[[ 0.0457, -0.1455,  0.1562,  ..., -0.0159,  0.0067, -0.0019],
         [ 0.0540,  0.1152, -0.1689,  ..., -0.4414, -0.0576,  0.2139],
         [-0.0679,  0.0952,  0.0356,  ...,  0.1270, -0.1035,  0.0476],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0166,  0.0457, -0.1191,  ...,  0.0165,  0.2637,  0.0845],
         [ 0.1279,  0.0479,  0.1069,  ...,  0.0742, -0.0039,  0.0503],
         [-0.0476,  0.0815,  0.0457,  ..., -0.0557,  0.1050, -0.1084],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0713,  0.0693,  0.0757,  ..., -0.2598, -0.1050,  0.0103],
         [-0.2256, -0.0195,  0.0908,  ...,  0

### Modeling

In [26]:
from models import ClassificationModel

model = ClassificationModel(len(train_vocab),embedding_dim=EMBEDDING_DIM,hidden_dim = HIDDEN_DIM,num_layers = NUM_LAYERS,bidirectional = BIDIRECTIONAL)

model.to(device)

ClassificationModel(
  (LSTM): LSTM(300, 128, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=256, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

In the following cell, **instantiate the model with some hyperparameters, and select an appropriate loss function and optimizer.** 

Hint: we already use sigmoid in our model. What loss functions are availible for binary classification? Feel free to look at PyTorch docs for help!

In [28]:
from torch.optim import AdamW

criterion, optimizer = torch.nn.CrossEntropyLoss(), torch.optim.Adam(model.parameters(), lr=0.001)

### Part 3: Training and Evaluation [10 Points]
The final part of this HW involves training the model, and evaluating it at each epoch. **Fill out the train and test loops below.**

In [30]:
# returns the total loss calculated from criterion
def train_loop(model, criterion, iterator):
    model.train()
    total_loss = 0
    
    for x, y in tqdm(iterator):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        prediction = model(x)
        prediction = torch.squeeze(prediction)
        # y = y.round()
        y = y.long()
        

 
        loss = criterion(prediction,y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    return total_loss

# returns:
# - true: a Python boolean array of all the ground truth values 
#         taken from the dataset iterator
# - pred: a Python boolean array of all model predictions. 
def val_loop(model, criterion, iterator):
    true, pred = [], []
    for x, y in tqdm(iterator):
        x = x.to(device)
        y = y.to(device)
    
        preds = model(x)
        preds.to(device)
        preds = torch.squeeze(preds)
        for i_batch in range(len(y)):
            true.append(y[i_batch])
            pred.append(torch.argmax(preds[i_batch]))
            
    return true, pred


In [32]:
# To test your eval implementation, let's see how well the untrained model does on our dev dataset.
# It should do pretty poorly.
from sklearn.metrics import f1_score, accuracy_score

from eval_utils import binary_macro_f1, accuracy
true, pred = val_loop(model, criterion, val_iterator)
true = [x.item() for x in true]
pred = [x.item() for x in pred]
print(f1_score(true, pred, average='weighted'))
print(accuracy_score(true, pred))


100%|██████████| 123/123 [00:04<00:00, 27.39it/s]


0.14270770601918598
0.29943791517629026


### Actually training the model

In [36]:
TOTAL_EPOCHS = 10
for epoch in range(TOTAL_EPOCHS):
    train_loss = train_loop(model, criterion, train_iterator)
    true, pred = val_loop(model, criterion, val_iterator)
    true = [x.item() for x in true]
    pred = [x.item() for x in pred]
    print(f"EPOCH: {epoch}")
    print(f"TRAIN LOSS: {train_loss}")
    print(f"VAL F-1: {f1_score(true, pred, average='weighted')}")
    print(f"VAL ACC: {accuracy_score(true, pred)}")


100%|██████████| 123/123 [00:04<00:00, 27.14it/s]


EPOCH: 0
TRAIN LOSS: 288.6155935116112
VAL F-1: 0.7541545124133647
VAL ACC: 0.7557485947879408


100%|██████████| 123/123 [00:04<00:00, 27.08it/s]


EPOCH: 1
TRAIN LOSS: 219.90362956933677
VAL F-1: 0.7461215676959659
VAL ACC: 0.747061829330608


100%|██████████| 123/123 [00:04<00:00, 27.25it/s]


EPOCH: 2
TRAIN LOSS: 167.11705572437495
VAL F-1: 0.7544326303088776
VAL ACC: 0.7547266223811957


100%|██████████| 123/123 [00:04<00:00, 27.38it/s]


EPOCH: 3
TRAIN LOSS: 121.28197964397259
VAL F-1: 0.747481293705993
VAL ACC: 0.7480838017373531


 22%|██▏       | 213/979 [00:26<01:35,  8.01it/s]


KeyboardInterrupt: ignored

We can also look at the models performance on the held-out test set, using the same val_loop we wrote earlier.

In [37]:
true, pred = val_loop(model, criterion, test_iterator)
true = [x.item() for x in true]
pred = [x.item() for x in pred]
print(f"TEST F-1: {f1_score(true, pred, average='weighted')}")
print(f"TEST ACC: {accuracy_score(true, pred)}")

100%|██████████| 123/123 [00:04<00:00, 27.56it/s]


TEST F-1: 0.7464677778562899
TEST ACC: 0.7475728155339806
